# Reddit PRAW scraper for NHL Talk

In [14]:
#import all the stuff
import praw
import pandas as pd
import datetime as dt
import config as c

## Save all the praw codes in one thing called reddit

In [15]:
reddit = praw.Reddit(
    client_id= c.client_id,
    client_secret = c.client_secret,
    user_agent = c.user_agent,
    username= c.username,
    password= c.password
)

## Load all subreddit submissions into  list and turn that into a dataframe

In [16]:
#save opening day 2018 as a unix timestamp
opening_day_2018 = pd.to_datetime('2018-10-03')

posts = [] #create empty list for posts

#look up the "nhl" subreddit on opening day 2018 and search extra terms
nhl_subreddit = reddit.subreddit('sportsbook')


#use a slick loop to load each entry and its elements into you "posts" list
    #note: just change the limit to anything under 1,000 and you'll get tons of things
for post in nhl_subreddit.search('hockey',sort='new', time_filter='day',limit=5):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])

#turn posts into a DataFrame
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [17]:
#Change 'created' column from Unix to datetime so it looks normal
posts['created'] = pd.to_datetime(posts['created'], unit='s')
#print a bit of the df to see if it looks ok
posts.head()

,title,score,id,subreddit,url,num_comments,body,created
0,NHL Daily Discussion - 3/19/21 (Friday),18,m877ox,sportsbook,https://www.reddit.com/r/sportsbook/comments/m...,25,For NHL **and** world hockey. Hockey Chat: htt...,2021-03-19 10:30:56


## Take all the comments from these submissions and throw them into a separate dataframe

In [18]:
#get all comments from each submission and make into a list
comments_list = [] 
for url in posts.url:
    #get save submission title as variable
    submission_title = posts.loc[posts['url'] == url,'title'].to_string(index=False)
    
    #then get all the comments
    submission = reddit.submission(url=url)
    submission.comments.replace_more(limit=0)
    

    for top_level_comment in submission.comments.list():
        comments_list.append([submission_title, top_level_comment.submission, top_level_comment.score, top_level_comment.id, top_level_comment.subreddit, 'https://www.reddit.com' + top_level_comment.permalink, top_level_comment.replies, top_level_comment.body, top_level_comment.created_utc])
    
comments_df = pd.DataFrame(comments_list,columns=['submission_title', 'submission_id', 'score', 'comment_id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
#change 'created' into datetime dates
comments_df['created'] = pd.to_datetime(comments_df['created'], unit='s')

comments_df.head()

,submission_title,submission_id,score,comment_id,subreddit,url,num_comments,body,created
0,NHL Daily Discussion - 3/19/21 (Friday),m877ox,1,grfs99g,sportsbook,https://www.reddit.com/r/sportsbook/comments/m...,(),Hockey Live Chat: **https://discord.gg/sportsb...,2021-03-19 02:30:56
1,NHL Daily Discussion - 3/19/21 (Friday),m877ox,3,grgitm5,sportsbook,https://www.reddit.com/r/sportsbook/comments/m...,(),"Maple Leafs ML, NY Rangers ML, Canadiens ML, S...",2021-03-19 07:50:45
2,NHL Daily Discussion - 3/19/21 (Friday),m877ox,4,grh2nlx,sportsbook,https://www.reddit.com/r/sportsbook/comments/m...,"(grh2t8f, gri3cy0)",Record: 8-4\n\nPick: **MTL Canadiens in Reg -1...,2021-03-19 12:42:32
3,NHL Daily Discussion - 3/19/21 (Friday),m877ox,4,grh8f3l,sportsbook,https://www.reddit.com/r/sportsbook/comments/m...,(),Capitals are on a win streak and generally pl...,2021-03-19 13:37:24
4,NHL Daily Discussion - 3/19/21 (Friday),m877ox,4,grhfu7b,sportsbook,https://www.reddit.com/r/sportsbook/comments/m...,(),**NHL Freezer Machine Learning 3-Way Model**\n...,2021-03-19 14:39:01
